In [21]:
%reset -f

import os
import numpy as np
import pandas as pd
import math
from math import e

import configparser

In [52]:
# lectura de datos
    
def load_data():
    """Lee los datos del directorio de trabajo.
    """
    global pathoutput
    global nodos, inv, vu, cartectraf
    global HSCD, PVDT, TNTR, TAMB, PDRE, FCMX, FCPR, HVUT, AVUT
    global P1FVAC1, P1FVAC2, P1FCAR1, P1FCAR2, P1FCAR3, P1FCAR4
    global P3FVAC11, P3FVAC12, P3FVAC21, P3FVAC22, P3FVAC31, P3FVAC32
    global P3FCAR11, P3FCAR12, P3FCAR13, P3FCAR21, P3FCAR22, P3FCAR31, P3FCAR32

    pathoutput = working_dir + 'output/'

    ## definicion del archivo txt donde estan los parametros
    parser = configparser.ConfigParser()
    parser.read(working_dir + 'input/params.txt')

    ## parametros globales
    HSCD = float(parser['CALCULOS']['horas_sobrecarga_dia'])
    PVDT = float(parser['CALCULOS']['perdida_vida_diaria_teorica'])
    TNTR = float(parser['CALCULOS']['temperatura_normal_trafo'])
    TAMB = float(parser['CALCULOS']['temperatura_ambiente'])
    PDRE = float(parser['CALCULOS']['probabilidad_deterioro_reubicacion'])
    FCMX = float(parser['CALCULOS']['factor_carga_maxima'])
    FCPR = float(parser['CALCULOS']['factor_carga_promedio'])

    ## calculos con parametros globales
    HVUT = 24 / PVDT
    AVUT = HVUT / (24*365) 

    ##
    ## Resoluciones 818 y 819
    ##

    ##    Transformadores monofasicos - Perdidas en vacio
    ##
    P1FVAC1 = float(parser['RES818819']['Par_1f_vac_1'])
    P1FVAC2 = float(parser['RES818819']['Par_1f_vac_2'])

    ##
    ##    Transformadores monofasicos - Perdidas con carga
    ##        
    P1FCAR1 = float(parser['RES818819']['Par_1f_car_1'])
    P1FCAR2 = float(parser['RES818819']['Par_1f_car_2'])
    P1FCAR3 = float(parser['RES818819']['Par_1f_car_3'])
    P1FCAR4 = float(parser['RES818819']['Par_1f_car_4'])

    ##
    ##    Transformadores trifasicos -- Perdidas en vacio
    ##
    P3FVAC11 = float(parser['RES818819']['Par_3f_vac_11'])
    P3FVAC12 = float(parser['RES818819']['Par_3f_vac_12'])
    P3FVAC21 = float(parser['RES818819']['Par_3f_vac_21'])
    P3FVAC22 = float(parser['RES818819']['Par_3f_vac_22'])
    P3FVAC31 = float(parser['RES818819']['Par_3f_vac_31'])
    P3FVAC32 = float(parser['RES818819']['Par_3f_vac_32'])

    ##
    ##    Transformadores trifasicos - Perdidas con carga
    ##        
    P3FCAR11 = float(parser['RES818819']['Par_3f_car_11'])
    P3FCAR12 = float(parser['RES818819']['Par_3f_car_12'])
    P3FCAR13 = float(parser['RES818819']['Par_3f_car_13'])
    P3FCAR21 = float(parser['RES818819']['Par_3f_car_21'])
    P3FCAR22 = float(parser['RES818819']['Par_3f_car_22'])
    P3FCAR31 = float(parser['RES818819']['Par_3f_car_31'])
    P3FCAR32 = float(parser['RES818819']['Par_3f_car_32'])

    ##
    ## tablas de datos
    ##
    nodos = pd.read_csv(working_dir + "input/nodos.csv", sep=',', decimal='.')
    inv = pd.read_csv(working_dir + "input/inventario_transformadores.csv", sep=',', decimal='.')
    cartectraf = pd.read_csv(working_dir + "input/carac_tecn_transf.csv", sep=',', decimal='.')
    vu = pd.read_csv(working_dir + "input/vida_util.csv", sep=',', decimal='.')

    ##
    ## nombres de las columnas de las tablas de datos
    ##
    nodos.columns = ['id_n', 'id_n_Internexa','lat','lon','tension','cpro_n','cmax_n','cremcreg','dmda_n','cens','cred','tusu','pkwh_n']
    inv.columns = ['id_t', 'id_t_Internexa','fab','fase_t','tais','capa_t','vprim','vsecu','ffab','anus','viut_t','id_n_Internexa','tacr_t','creu_t','finst']
    vu.columns = ['tgrc', 'fase_t','lipo','lspo','cpre','dura','cpor','tmpc','tmac']
    cartectraf.columns = ['fase_t', 'capa_t','cnue_t']

    ## 
    ## adecuacion de las tablas para facilidad en calculos
    ##
    inv = inv.merge(nodos[['id_n','id_n_Internexa']], on = 'id_n_Internexa',how = 'left')

    ## calcular carga maxima y carga promedio de los nodos
    nodos['cpro_n'] = nodos['dmda_n'] / 30 * FCPR
    nodos['cmax_n'] = nodos['dmda_n'] / 30 * FCMX

    ## indicar grupo del trafo para calculo de las perdidas de transformacion
    inv['grpt_t'] = 1
    inv.loc[(inv.fase_t == 3) & (inv.capa_t >= 150), 'grpt_t'] = 2
    inv.loc[(inv.fase_t == 3) & (inv.capa_t >= 800), 'grpt_t'] = 3

    ## indicar grupo del trafo para calculo de las perdidas de vida util
    inv['grpv_t'] = 1
    inv.loc[(inv.fase_t == 1) & (inv.capa_t > 50), 'grpv_t'] = 2
    inv.loc[(inv.fase_t == 3) & (inv.capa_t >= 150), 'grpv_t'] = 2
    inv.loc[(inv.fase_t == 3) & (inv.capa_t >= 500), 'grpv_t'] = 3

    ## calcular vida util restante del trafo en meses
    inv['viut_t'] = AVUT
    inv['viur_t'] = (inv.viut_t - inv.anus) * 12
    inv.loc[inv.viur_t < 0, 'viur_t'] = 1

    ## indicar grupo de vida util
    vu['grpv_t'] = 1
    vu.loc[(vu.fase_t == 1) & (vu.lipo >= 50), 'grpv_t'] = 2
    vu.loc[(vu.fase_t == 3) & (vu.lipo >= 150),'grpv_t'] = 2
    vu.loc[(vu.fase_t == 3) & (vu.lipo >= 500), 'grpv_t'] = 3

    ## armar keys para busquedas
    cartectraf['faca'] = cartectraf.fase_t.map(str) + "-" + cartectraf.capa_t.map(str)
    vu['tfcg'] = vu.tgrc.map(str) + "-" + vu.fase_t.map(str) + "-" + vu.cpre.map(str) + "-" + vu['grpv_t'].map(str)


In [53]:
# funciones core

def parnd(id_n):
    """Obtiene los parametros de un nodo.

    Args:
        id_n (int): id del nodo.

    Returns:
        cmax_n (float): carga maxima que soporta el nodo.
        cpro_n (float): carga promedio que soporta el nodo.
        pkwh_n (float): precio por kWh de la electricidad que sirve el nodo.

    """
    cmax_n = float(nodos[nodos.id_n == id_n]['cmax_n'])
    cpro_n = float(nodos[nodos.id_n == id_n]['cpro_n'])    
    pkwh_n = float(nodos[nodos.id_n == id_n]['pkwh_n']) 
    return (cmax_n,cpro_n,pkwh_n)


def partf(id_t):
    """Obtiene los parametros de un trafo.

    Args:
        id_t (int): id del trafo.

    Returns:
        capa_t (float): capacidad del trafo.
        fase_t (int): numero de fases del trafo.
        viut_t (int): vida utuil teorica del trafo.
        nodo_t (int): nodo al que se encuentra asociado el trafo.
        creu_t (float): costo de la actividad de reubicacion del trafo.
        viur_t (float): vida util restante del trafo.
        grpt_t (int): grupo al que pertenece el trafo para el calculo de las perdidas de transformacion.
        grpv_t (int): grupo al que pertenece el trafo para el calculo de las perdidas de vida util.
        faca_t (str): key fase-capacidad.
        cnue_t (int): precio por kWh de la electricidad que sirve el nodo.

    """
    capa_t = float(inv[inv.id_t == id_t]['capa_t'])
    fase_t = int(inv[inv.id_t == id_t]['fase_t'])
    viut_t = int(inv[inv.id_t == id_t]['viut_t'])
    nodo_t = int(inv[inv.id_t == id_t]['id_n'])
    creu_t = float(inv[inv.id_t == id_t]['creu_t'])
    viur_t = float(inv[inv.id_t == id_t]['viur_t'])
    grpt_t = int(inv[inv.id_t == id_t]['grpt_t'])
    grpv_t = int(inv[inv.id_t == id_t]['grpv_t'])
    faca_t = str(fase_t) + '-' + str(capa_t)
    cnue_t = int(cartectraf[cartectraf.faca == faca_t]['cnue_t'])
    return (capa_t, fase_t, viut_t, nodo_t, creu_t, viur_t, grpt_t, grpv_t, faca_t, cnue_t)


def cospt(id_n, id_t):
    """Calcula los costos de las perdidas de transformacion de un par nodo*trafo.

    Args:
        id_n (int): id del nodo.
        id_t (int): id del trafo.

    Returns:
        cpt_nt (float): costos de perdidas de transformacion en pesos.

    """
    if id_n == 999999:
        cpt_nt = 0; cpfe_nt = 0; cpcu_nt = 0
    else:
        # hallar parametros del nodo y del trafo
        cmax_n,cpro_n,pkwh_n = parnd(id_n)
        capa_t,fase_t,viut_t,nodo_t,creu_t,viur_t,grpt_t,grpv_t,faca_t,cnue_t = partf(id_t)
        futi_nt = cmax_n / capa_t

        # calcular perdidas nominales en vacio y perdidas nominales con carga en funcion de las fases y el grupo en la fase
        if fase_t == 1:
            pnvac = P1FVAC1 * capa_t ** P1FVAC2
            pncar = P1FCAR1 * capa_t ** 3 + P1FCAR2 * capa_t ** 2 + P1FCAR3 * capa_t + P1FCAR4
        else:
            if grpt_t == 1:
                pnvac = P3FVAC11 * capa_t ** P3FVAC12
                pncar = P3FCAR11 * capa_t ** 2 + P3FCAR12 * capa_t + P3FCAR13
            if grpt_t == 2:
                pnvac = P3FVAC21 * capa_t ** P3FVAC22
                pncar = P3FCAR21 * capa_t + P3FCAR22
            if grpt_t == 3:
                pnvac = P3FVAC31 * capa_t ** P3FVAC32
                pncar = P3FCAR31 * capa_t + P3FCAR32

        # calcular las perdidas en hierro y cobre en unidades W 
        pfeW = pnvac
        pcuW = pncar * futi_nt ** 2
        ptrW = pfeW + pcuW

        # monetizacion de las perdidas
        cpfe_nt = pfeW / 1000 * pkwh_n * 24 * 30 * viur_t / viur_t
        cpcu_nt = pcuW / 1000 * pkwh_n * 24 * 30 * viur_t / viur_t
        cpt_nt = ptrW / 1000 * pkwh_n * 24 * 30 * viur_t / viur_t
    return (cpt_nt, cpfe_nt, cpcu_nt)


def cosvu(id_n, id_t):
    """Calcula los costos de las perdidas de vida util de un par nodo*trafo

    Args:
        id_n (int): id del nodo.
        id_t (int): id del trafo.

    Returns:
        cpt_vu (float): costos de perdidas de vida util en pesos.

    """
    if id_n == 999999:
        cvu_nt = 0; viur_r = 0
    else:
        # hallar parametros del nodo y del trafo
        cmax_n,cpro_n,pkwh_n = parnd(id_n)
        capa_t,fase_t,viut_t,nodo_t,creu_t,viur_t,grpt_t,grpv_t,faca_t,cnue_t = partf(id_t)

        # calcular la carga precedente y el factor de utilizacion
        cpre_nt = cpro_n / capa_t
        futi_nt = cmax_n / capa_t

        # aproximar la carga precedente a los valores de la norma GTC50
        if cpre_nt < ((0.5 + 0.75) / 2):
            cpre_nt = 0.5
        else:
            if cpre_nt < ((0.75 + 0.9) / 2):
                cpre_nt = 0.75
            else:
                cpre_nt = 0.9

        # calcular el porcentaje diario de perdida de vida util real en porcentaje
        key = str(int(TAMB)) + '-' + str(fase_t) + '-' + str(cpre_nt) + '-' + str(grpv_t)
        theta = temperPC(futi_nt,key)
        fevej = (HSCD / 24) * (e**(15000/383 - 15000/(theta + 273))-1)
        pvdr = PVDT * (1 + fevej)

        # valorar perdida de vida util restante en pesos, durante lo que queda de vida util del trafo en el nodo 
        cvu_nt = viur_t * 30 * cnue_t * pvdr / viur_t
        viur_r = 12 / (pvdr * 365)
    return (cvu_nt, viur_r)


def temperPC(futi_nt, key):
    """Calcula la temperatura del punto mas caliente dado un factor de utilizacion

    Args:
        futi_nt (float): factor de utilizacion.
        key (int): key TAMB - fase_t - cpre_nt - grpv_t.

    Returns:
        theta (int): temperatura del punto mas caliente.

    """
    tabvu = vu.loc[(vu.tfcg == str(key)) & (vu.dura <= HSCD)]
    ncargas = tabvu.shape[0]
    carga=futi_nt * 100
    theta=0
    if carga < tabvu['cpor'].min(): theta = TNTR
    if carga >= tabvu['cpor'].max(): theta = tabvu['tmpc'].max()
    if theta == 0:
        tabvu = tabvu.sort_values(['cpor'],ascending=[False])
        for index, row in tabvu.iterrows():
            if carga <= row['cpor']:
                theta = row['tmpc']
                break
    return theta 


def costopermtf(id_t):
    """Calcula los costos de permutacion del trafo

    Args:
        id_t (int): id del trafo.

    Returns:
        cperm_t (float): costo de permutacion del trafo.

    """
    viur_t = float(inv[inv.id_t == id_t]['viur_t'])
    cperm_t = float(inv[inv.id_t == id_t]['creu_t']) / viur_t
    return cperm_t


def costodetetf(id_t):
    """Calcula los costos de deterioro de un trafo por reubicacion.

    Args:
        id_t (int): id del trafo.

    Returns:
        cdete_t (float): costo de deterioro del trafo.

    """
    cnue_t = partf(id_t)[9]
    viur_t = float(inv[inv.id_t == id_t]['viur_t'])
    cdete_t = cnue_t * PDRE / viur_t
    return cdete_t


In [54]:
#funciones comunes a las permutaciones

def cospermpar(id_t1, id_t2, sol_prov):
    """Calcula los costos de permutacion de dos trafos en una solucion provisional respecto a una solucion inicial.

    Args:
        id_t1 (int): id del trafo 1.
        id_t2 (int): id del trafo 2.
        sol_prov (tupla): solucion provisional de la red, asumiendo que la permutacion se realizo.

    Returns:
        cperm_t1 (float): costo de permutar el trafo 1, en pesos.
        cdete_t1 (float): costo de deterioro del trafo 1, en pesos.
        cperm_t2 (float): costo de permutar el trafo 2, en pesos.
        cdete_t1 (float): costo de deterioro del trafo 2, en pesos.

    """
    cperm_t1 = 0; cperm_t2 = 0; cdete_t1 = 0; cdete_t2 = 0

    # para el trafo 1
    id_n1_orig = int(solini[(solini.id_t == id_t1)]['id_n'])
    id_n1_prov = int(sol_prov[(sol_prov.id_t == id_t1)]['id_n'])
    if id_n1_orig != id_n1_prov:
        cperm_t1, cdete_t1 = costopermtf(id_t1), costodetetf(id_t1)

    # para el trafo 2
    id_n2_orig = int(solini[(solini.id_t == id_t2)]['id_n'])
    id_n2_prov = int(sol_prov[(sol_prov.id_t == id_t2)]['id_n'])
    if id_n2_orig != id_n2_prov:
        cperm_t2, cdete_t2 = costopermtf(id_t2), costodetetf(id_t2)
    return (cperm_t1, cdete_t1, cperm_t2, cdete_t2)


def bodega():
    """Calcula la bodega.
    """
    global bodini, bodfin
    
    bodini = solini[solini.id_n == 999999]
    bodfin = solfin[solfin.id_n == 999999]


In [55]:
# funcion para armar la solucion inicial
def armarsolini():
    """Arma la solucion actual de la red con sus costos asociados.
    """
    global solini, ctsolini, sol
    solini = pd.DataFrame(columns=['id_n','id_t','cpt','cpfe','cpcu','cvu','viur_r','coper','cperm','cdete','eval','perm','bloq'])

    for index, row in inv.iterrows():
        id_n = int(row['id_n'])
        id_t = int(row['id_t'])

        # calculos de costos para los nodos reales y en bodega
        cpt_nt = 0; cpfe_nt = 0; cpcu_nt = 0; cvu_nt = 0; viur_r = 0
        if id_n != 999999: 
            cpt_nt, cpfe_nt, cpcu_nt = cospt (id_n,id_t)
            cvu_nt, viur_r = cosvu (id_n,id_t)
        coper_nt = cpt_nt + cvu_nt

        # almacenar el costo para ese arreglo en particular
        solini.loc[index]= [id_n,id_t,cpt_nt,cpfe_nt,cpcu_nt,cvu_nt,viur_r,coper_nt,0,0,0,0,0]

    # poner carga maxima de nodos y capacidades de trafos en la solucion inicial
    solini = solini.merge(inv[['id_t','capa_t']], on = 'id_t',how = 'left')
    solini = solini.merge(nodos[['id_n','cmax_n']], on = 'id_n',how = 'left')

    # escribir la solucion inicial y calcular su costo
    solini.to_csv(pathoutput + 'solucion_inicial.csv')
    ctsolini = solini['coper'].sum()
    sol = solini.copy()


In [87]:
# funciones propias de la heuristica

def ndmascostoso ():
    """Elige el nodo mas costoso aun no evaluado, junto con su trafo asociado.

    Args:
        sol (tupla): solucion actual de la red.

    Returns:
        id_n (int): id del nodo mas costoso.
        id_t (int): id del trafo asociado al nodo mas costoso.

    """
    # seleccionar los nodos potenciales
    solcopia = sol.copy()
    solcopia = solcopia[solcopia['eval'] == 0]
    solcopia = solcopia.sort_values(['coper'],ascending=[False])

    id_n = int(solcopia.iloc[0,0])
    id_t = int(solcopia.iloc[0,1])
    sol.at[sol.id_n == id_n,'eval'] = 1
    return (id_n, id_t)


def tfmascostoso (id_n):
    """Elige el trafo mas costoso para hacer permutacion con un cierto nodo.

    Args:
        id_n (int): id del nodo.
        sol (tupla): solucion actual de la red.

    Returns:
        id_n2 (int): id del nodo asociado al trafo mas costoso.
        id_t2 (int): id del trafo mas costoso.

    """
    
    cmax_n = parnd(id_n)[0]
    id_t = int(sol[sol.id_n == id_n]['id_t'])
    capa_t = partf(id_t)[0]

    # seleccionar los transformadores potenciales
    solcopia = sol.copy()
    solcopia = solcopia[solcopia.id_n != id_n]
    solcopia = solcopia[solcopia['perm'] == 0]
    solcopia = solcopia[solcopia.capa_t <= cmax_n / 0.4]
    solcopia = solcopia.sort_values(['coper'],ascending=[False])

    # seleccionar el mas costoso de los nodos de los trafos
    id_n2=0; id_t2=0
    if solcopia.shape[0] > 0:
        id_n2 = int(solcopia.iloc[0,0])
        id_t2 = int(solcopia.iloc[0,1])
    return (id_n2, id_t2)


def condparada():
    """Verifica si se ha cumplido la condición de parada de las permutaciones.

    Args:
        sol (tupla): solucion actual de la red.

    Returns:
        stop (int): 1: se debe par, 0: se debe continuar.

    """
    
    global stop
    stop = 0
    solcopia = sol.copy()
    solcopia = solcopia.loc[(sol['eval'] == 0) & (sol['coper'] > 0)]
    if solcopia.shape[0] == 0: stop = 1

In [91]:
def permutar():
    """Hace una permutacion de un par de trafos en una solucion y encuentra los costos asociados.

    Args:
        sol (tupla): solucion actual de la red.

    """
    
    global sol,tperm, solfin, cosolfin, cpsolfin, cdsolfin, ctsolfin
    
    # elegir el nodo mas costoso y el trafo mas costoso para el intercambio
    id_n1, id_t1 = ndmascostoso()
    id_n2, id_t2 = tfmascostoso(id_n1)
    while (id_t2 == 0 and id_n1 != 999999):
        id_n1, id_t1 = ndmascostoso()
        if id_n1 != 999999: id_n2, id_t2 = tfmascostoso(id_n1)
            
    if id_t2 != 0:
        # calcular los costos de operacion actuales en los dos nodos
        coact_n1 = 0; coact_n2 = 0
        if id_n1 != 999999: coact_n1 = float(sol[sol.id_n == id_n1]['coper'])
        if id_n2 != 999999: coact_n2 = float(sol[sol.id_n == id_n2]['coper'])
        coact = coact_n1 + coact_n2

        # calcular los costos de operacion despues de la permutacion en los dos nodos
        cpt_n1t2, cpfe_n1t2, cpcu_n1t2 = cospt(id_n1,id_t2)
        cpt_n2t1, cpfe_n2t1, cpcu_n2t1 = cospt(id_n2,id_t1)
        cvu_n1t2, viur_r12 = cosvu (id_n1,id_t2)
        cvu_n2t1, viur_r21 = cosvu (id_n2,id_t1)
        coper = cpt_n1t2 + cvu_n1t2 + cpt_n2t1 + cvu_n2t1

        # calcular los costos de permutacion de los transformadores
        sol_prov = sol.copy()
        sol_prov.at[(sol.id_n == id_n1) & (sol.id_t == id_t1),'id_t']=id_t2
        sol_prov.at[(sol.id_n == id_n2) & (sol.id_t == id_t2),'id_t']=id_t1
        cperm_t1,cdete_t1,cperm_t2,cdete_t2 = cospermpar(id_t1,id_t2,sol_prov)
        cperm = cperm_t1 + cperm_t2
        cdeter = cdete_t1 + cdete_t2

        # hacer el cambio oficial en la solucion
        if coact >= (coper + cperm + cdeter):

            # traer las capacidades
            capa_t1 = partf(id_t1)[0]
            capa_t2 = partf(id_t2)[0]

            #Actualizar la matriz de solución
            sol.at[(sol.id_n == id_n1) & (sol.id_t == id_t1),'id_t']=id_t2
            sol.at[(sol.id_n == id_n2) & (sol.id_t == id_t2),'id_t']=id_t1

            sol.at[(sol.id_n == id_n1) & (sol.id_t == id_t2),'cpt']=cpt_n1t2
            sol.at[(sol.id_n == id_n2) & (sol.id_t == id_t1),'cpt']=cpt_n2t1

            sol.at[(sol.id_n == id_n1) & (sol.id_t == id_t2),'cpfe']=cpfe_n1t2
            sol.at[(sol.id_n == id_n2) & (sol.id_t == id_t1),'cpfe']=cpfe_n2t1

            sol.at[(sol.id_n == id_n1) & (sol.id_t == id_t2),'cpcu']=cpcu_n1t2
            sol.at[(sol.id_n == id_n2) & (sol.id_t == id_t1),'cpcu']=cpcu_n2t1

            sol.at[(sol.id_n == id_n1) & (sol.id_t == id_t2),'viur_r']=viur_r12
            sol.at[(sol.id_n == id_n2) & (sol.id_t == id_t1),'viur_r']=viur_r21

            sol.at[(sol.id_n == id_n1) & (sol.id_t == id_t2),'cpcu']=cpcu_n1t2
            sol.at[(sol.id_n == id_n2) & (sol.id_t == id_t1),'cpcu']=cpcu_n2t1

            sol['coper'] = sol['cpt'] + sol['cvu']

            sol.at[(sol.id_n == id_n1) & (sol.id_t == id_t2),'cperm']=cperm_t2
            sol.at[(sol.id_n == id_n2) & (sol.id_t == id_t1),'cperm']=cperm_t1

            sol.at[(sol.id_n == id_n1) & (sol.id_t == id_t2),'cdete']=cdete_t2
            sol.at[(sol.id_n == id_n2) & (sol.id_t == id_t1),'cdete']=cdete_t1

            sol.at[(sol.id_n == id_n1) & (sol.id_t == id_t2),'capa_t']=capa_t2
            sol.at[(sol.id_n == id_n2) & (sol.id_t == id_t1),'capa_t']=capa_t1

            sol.at[(sol.id_t == id_t1),'perm']=1
            sol.at[(sol.id_t == id_t2),'perm']=1

            #Guardar estadísticas
            tperm += 1
            solfin = sol.copy() 
            cosolfin = sol['coper'].sum()
            cpsolfin = sol['cperm'].sum()
            cdsolfin = sol['cdete'].sum()
            ctsolfin = cosolfin + cpsolfin + cdsolfin


In [95]:
def run():
    """Ejecuta la rutina de optimizacion.
    """
    global working_dir 
    working_dir="../Tests/Test3/"
    
    global sol, titer, tperm
    
    load_data()
    armarsolini()
    # armar la matriz de progresos en el hallazgo de mejores soluciones
    prog_sol = pd.DataFrame(columns=['iteracion','tperm','coper','cperm','cdete','costo_total'])
    prog_sol.loc[0] = [0,0,ctsolini,0,0,ctsolini]

    # inicializar contadores y condicion de parada
    titer = 0; tperm = 0
    sol = solini.copy()
    condparada()

    # iterar
    while (stop == 0):
        # permutar
        permutar()

        # guardar progreso de la simulacion
        prog_sol.loc[titer] = [titer,tperm,cosolfin,cpsolfin,cdsolfin,ctsolfin]     

        # verificar condicion de parada
        condparada()

    # exportar los resultados de la solucion final
    bodega()
    solfin.to_csv(pathoutput + 'solucion_final.csv')
    prog_sol.to_csv(pathoutput + 'progreso_soluciones.csv')
    bodini.to_csv(pathoutput + 'bodega_inicial.csv')
    bodfin.to_csv(pathoutput + 'bodega_final.csv')


In [96]:
run()
print(ctsolini)
print(ctsolfin)


857371.6298163689
786227.5727945647


In [50]:
tperm

0